##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

# 装载nbfunc中的hub涉及的所有ipynb函数
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb

initGit()

%cd ..

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    "iputils-ping",
]

def extract_library_name(library):
    """从库名中提取出包的名称"""
    return library.split("==")[0]

def get_installed_libraries():
    """获取已安装的库列表"""
    output = subprocess.check_output(["pip", "freeze"])
    installed_libraries = output.decode().split("\n")
    return set([extract_library_name(library) for library in installed_libraries])

def install_library(library):
    """安装指定的库"""
    !pip install $library --user

# 获取已安装的库列表
installed_libraries = get_installed_libraries()

# 找出未安装的库并安装
libraries_to_install_filtered = [library for library in libraries_to_install if extract_library_name(library) not in installed_libraries]
if libraries_to_install_filtered:
    print(f"以下库将被安装：{libraries_to_install_filtered}")
    for library in libraries_to_install_filtered:
        install_library(library)
else:
    print("所有需要安装的库都已经安装。")

# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

# 检测Clash
检测Clash是否可以运行，如果不行，则进行下载，并解压

In [ ]:
import os
import requests
import time
import glob  # 导入 glob 模块

def get_latest_clash_download_url(retries=1, backoff=1):
    url = 'https://api.github.com/repos/Dreamacro/clash/releases/latest'

    for _ in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            json_data = response.json()
            for asset in json_data['assets']:
                if 'clash-linux-amd64' in asset['name']:
                    return asset['browser_download_url'], json_data['tag_name']
            return '', ''
        except requests.exceptions.RequestException as e:
            print(f"请求出错: {e}")
            time.sleep(backoff)
    else:
        print("达到最大重试次数，获取最新版本下载链接失败")
        return '', ''

def download_clash_installer(download_url, root_dir, clash_install_dir):
    file_name = download_url.split('/')[-1]
    local_path = os.path.join(root_dir, clash_install_dir, file_name)

    os.makedirs(os.path.join(root_dir, clash_install_dir), exist_ok=True)

    if not os.path.exists(local_path):
        os.system(f'wget -P {os.path.join(root_dir, clash_install_dir)} {download_url}')
        print(f"已下载文件 {file_name}")
    else:
        print(f"文件 {file_name} 已存在，无需下载")

    return local_path

def extract_clash_installer(local_path, root_dir, clash_run_dir, latest_version):
    run_path = os.path.join(root_dir, clash_run_dir, os.path.basename(local_path))
    os.makedirs(os.path.join(root_dir, clash_run_dir), exist_ok=True)

    os.system(f'cp {local_path} {run_path}')
    os.system(f'gunzip {run_path}')

    clash_exec = os.path.join(root_dir, clash_run_dir, f'clash-linux-amd64-{latest_version}')
    clash_sim_exec = os.path.join(root_dir, clash_run_dir, 'clash')

    os.system(f'chmod +x {clash_exec}')
    os.system(f'mv {clash_exec} {clash_sim_exec}')

    return clash_sim_exec

def check_clash_executable(root_dir, clash_run_dir):
    clash_sim_exec = os.path.join(root_dir, clash_run_dir, 'clash')
    if os.path.exists(clash_sim_exec) and os.access(clash_sim_exec, os.X_OK):
        print("Clash已经安装,并可以执行。")
        return True
    elif os.path.exists(clash_sim_exec):
        print("Clash已经安装,但不可以执行，即将重装。")
        os.remove(clash_sim_exec)
        return False
    else:
        return False

def download_and_install_clash(root_dir, clash_install_dir, clash_run_dir):
    if check_clash_executable(root_dir, clash_run_dir):
        return

    clash_installer_prefix = 'clash-linux-amd64-'
    clash_installer_suffix = '.gz'
    clash_installer_pattern = clash_installer_prefix + '*' + clash_installer_suffix
    clash_installer_glob = os.path.join(root_dir, clash_install_dir, clash_installer_pattern)
    clash_installer_paths = sorted(glob.glob(clash_installer_glob), reverse=True)

    if clash_installer_paths:
        clash_installer_path = clash_installer_paths[0]
        clash_installer_name = os.path.basename(clash_installer_path)
        clash_version = clash_installer_name[len(clash_installer_prefix):-len(clash_installer_suffix)]
        print(f"发现 {clash_installer_name} 文件，将进行解压并安装。")
        clash_sim_exec = extract_clash_installer(clash_installer_path, root_dir, clash_run_dir, clash_version)
    else:
        download_url, latest_version = get_latest_clash_download_url()
        if not download_url:
            print('获取最新版本下载链接失败')
            return
        clash_version = latest_version
        file_name = f"{clash_installer_prefix}{clash_version}{clash_installer_suffix}"
        local_path = download_clash_installer(download_url, root_dir, clash_install_dir)
        clash_sim_exec = extract_clash_installer(local_path, root_dir, clash_run_dir, clash_version)

    if not check_clash_executable(root_dir, clash_run_dir):
        print("Clash安装失败，请检查权限等问题。")
        return

    print("Clash已经安装。")

clash_install_dir = "clashInstall"
clash_run_dir = "clash_run"

download_and_install_clash(root_dir, clash_install_dir, clash_run_dir)

In [ ]:
import os
import re
    
def download_config(root_dir, config_name, config_dir, mmdb_name, mmdb_dir, clash_install_dir, config_yaml_dist,origin_yaml_dist):
    clash_install_path = os.path.join(root_dir, clash_install_dir)
    mmdb_path = os.path.join(root_dir, mmdb_dir)
    config_path = os.path.join(root_dir, config_dir)

    # 下载Country.mmdb文件
    check_mmdb_dist = os.path.join(clash_install_path, mmdb_name)
    if not os.path.isfile(check_mmdb_dist):
        !wget -O {check_mmdb_dist} https://www.sub-speeder.com/client-download/{mmdb_name}
    country_mmdb_dist = os.path.join(mmdb_path, mmdb_name)
    if not os.path.isfile(country_mmdb_dist):
        !mkdir -p {mmdb_path}
        !cp {check_mmdb_dist} {country_mmdb_dist}
        print("Country.mmdb已经复制到配置目录。")
    else:
        print("Country.mmdb已经下载。")

    # 下载和复制config.yaml文件
    import shlex

    origin_yaml_dist = shlex.quote(origin_yaml_dist)
    config_yaml_dist = shlex.quote(config_yaml_dist)
    
    
    if not os.path.isfile(config_yaml_dist):
        !wget -O {config_yaml_dist} "https://raw.githubusercontent.com/Dreamacro/clash-dashboard/master/docker/clash-configs/{config_name}"
    if not os.path.isfile(config_yaml_dist):
        !mkdir -p {config_path}
        !cp {origin_yaml_dist} {config_yaml_dist}
        print("config.yaml已经复制到配置目录。")
    else:
        !cp {origin_yaml_dist} {config_yaml_dist}
        print("config.yaml已被覆盖。")

    print("Clash配置已经完成。")

import yaml

def modify_yaml_config(config_file_path, key_values_dict):
    try:
        with open(config_file_path, 'r') as f:
            config = yaml.load(f, Loader=yaml.SafeLoader)
        for key, value in key_values_dict.items():
            keys = key.split('.')
            sub_config = config
            for sub_key in keys[:-1]:
                sub_config = sub_config[sub_key]
            sub_config[keys[-1]] = value
        with open(config_file_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
    except yaml.parser.ParserError as e:
        print("\033[1;41;30m" + f"无法解析 YAML 配置文件：{e}。" + "\033[0m")
        print("\033[1;41;30m" + "请检查 YAML 文件的内容是否正确。" + "\033[0m")
    except (FileNotFoundError, KeyError, TypeError) as e:
        print("\033[1;41;30m" + f"无法修改 YAML 配置文件：{e}。" + "\033[0m")
        print("\033[1;41;30m" + "请检查指定的键或值是否有效。" + "\033[0m")

def yaml_config_clean(yaml_file_path):
    # 读取配置文件
    with open(yaml_file_path, 'r') as f:
        config = yaml.safe_load(f)

    # 清除带有 RULE-SET 的规则
    if 'rules' in config:
        rules = config['rules']
        config['rules'] = [rule for rule in rules if not rule.startswith('RULE-SET')]

    # 保存配置文件
    with open(yaml_file_path, 'w') as f:
        yaml.dump(config, f)

    print('Clash配置已经清理。')

import os
import sys
import subprocess
import threading
from IPython.display import display
from ipywidgets import Output

# Define color codes
COLOR_RESET = '\033[0m'
COLOR_CYAN = '\033[96m'
COLOR_YELLOW = '\033[93m'
COLOR_RED = '\033[91m'
COLOR_MAGENTA = '\033[95m'

def stream_output(process, output_widget):
    for line in iter(process.stdout.readline, b''):
        line = line.decode('utf-8')
        match = re.search('time="([^"]*)" level=([^ ]*) msg="([^"]*)"', line)
        if match:
            level = match.group(2)
            if level == 'info':
                level_str = COLOR_CYAN + '[INFO]' + COLOR_RESET
            elif level == 'warning':
                level_str = COLOR_YELLOW + '[WARNING]' + COLOR_RESET
            elif level == 'error':
                level_str = COLOR_RED + '[ERROR]' + COLOR_RESET
            else:
                level_str = level
            output_widget.append_stdout(level_str + ' ' + match.group(3) + '\n')
        else:
            output_widget.append_stdout(COLOR_MAGENTA + '[UNDEFINED]' + COLOR_RESET + ' ' + line + '\n')
    process.stdout.close()

def run_clash(root_dir, config_dir, clash_run_dir, config_yaml_dist):
    clash_run_path = os.path.join(root_dir, clash_run_dir)
    
    # 如果目录不存在，创建目录并更改权限
    if not os.path.exists(clash_run_path):
        os.makedirs(clash_run_path)
        os.chmod(clash_run_path, 0o777) # 设置目录权限为 777

    clash_Sim_EXEC = os.path.join(clash_run_path, 'clash')
    
    # !{clash_Sim_EXEC} -d {os.path.dirname(config_yaml_dist)}
    
    cmd = [clash_Sim_EXEC, '-d', os.path.dirname(config_yaml_dist)]

    print((' ').join(cmd))

    # 创建一个 Output 小部件
    output_widget = Output()
    display(output_widget)

    # 启动子进程并将 stdout 和 stderr 设置为 subprocess.PIPE，以便我们可以捕获输出
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # 创建一个线程来读取子进程的输出并将其发送到 Output 小部件
    output_thread = threading.Thread(target=stream_output, args=(process, output_widget))
    output_thread.start()


key_values_dict = {
    'external-controller': '127.0.0.1:7776',
    'port': 7777,
    'socks-port': 7778,
    'mixed-port': 7779,
    # 'mode':'Global'
}
    
config_name = "Dler Cloud.yaml"
config_dir = ".config/clash"
mmdb_name = "Country.mmdb"
mmdb_dir = ".config/clash"

origin_yaml_dist = os.path.join(root_dir, clash_install_dir, config_name)
config_yaml_dist = os.path.join(root_dir, config_dir, 'config.yaml')

download_config(root_dir, config_name, config_dir, mmdb_name, mmdb_dir, clash_install_dir,config_yaml_dist,origin_yaml_dist)
modify_yaml_config(config_yaml_dist, key_values_dict)
yaml_config_clean(config_yaml_dist)
run_clash(root_dir, config_dir, clash_run_dir,config_yaml_dist)